In [1]:
# Train your own tokenizer
# Acquiring a corpus
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# Build a generator
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
# Write to a text file
with open("../data/wikitext-2.txt", "w", encoding="utf-8") as f:
    for i in range(len(dataset)):
        f.write(dataset[i]["text"] + "\n")

In [5]:
# Building a WordPiece tokenizer
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
# Specify unknown token
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [6]:
# Set normalizer, using BertNormalizer
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [9]:
# Build BertNormalizer by hand
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()])

In [10]:
# Check out an example
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [15]:
# Pre-tokenize using prebuilt BertPreTokenizer
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [12]:
# Build one from scratch
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [16]:
# Test it out
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

In [23]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.Whitespace(), pre_tokenizers.Punctuation()])

# Assign to tokenizer
tokenizer.pre_tokenizer = pre_tokenizer
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

In [24]:
# Build a trainer
special_tokens = ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [25]:
# Train the tokenizer
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [27]:
# Test it out
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.']


In [28]:
# Get the IDs of special tokens
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

1 2


In [31]:
# Set BERT-style template
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [32]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '[SEP]']


In [33]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '.', '.', '[SEP]', 'on', 'a', 'pair', 'of', 'sentences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [34]:
# Add a decoder
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [35]:
# Test it out
tokenizer.decode(encoding.ids)

"let ' s test this tokenizer... on a pair of sentences."

In [36]:
# Save the tokenizer to disk
tokenizer.save("../data/wordpiece-tokenizer.json")


In [37]:
# Re-load our tokenizer
new_tokenizer = Tokenizer.from_file("../data/wordpiece-tokenizer.json")

In [38]:
# Wrap our tokenizer in a PreTrainedTokenizerFast
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=new_tokenizer,
    unk_token="[UNK]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    pad_token="[PAD]",
    mask_token="[MASK]",
)

In [ ]:
# Build a BPE tokenizer from scratch
bpe_tokenizer = Tokenizer(models.BPE())

# No need for normalizer

# Add pre-tokenizer (no space prefix)
bpe_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
bpe_tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my BPE tokenizer.")

# Build a trainer
bpe_trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
bpe_tokenizer.train_from_iterator(get_training_corpus(), trainer=bpe_trainer)





['L', 'et', "'", 's', 'Ġtest', 'Ġmy', 'ĠBP', 'E', 'Ġto', 'ken', 'izer', '.']


In [44]:
# Add post-processor
bpe_tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
sentence = "Let's test my BPE tokenizer."
bpe_encoding = bpe_tokenizer.encode(sentence)
print(bpe_encoding.tokens)
print(bpe_encoding.offsets)
start, end = bpe_encoding.offsets[4]
print(sentence[start:end])

# Add a byte-level decoder
bpe_tokenizer.decoder = decoders.ByteLevel()
bpe_tokenizer.decode(bpe_encoding.ids)

# Wrap in PreTrainedTokenizerFast
bpe_wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=bpe_tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)

['L', 'et', "'", 's', 'Ġtest', 'Ġmy', 'ĠBP', 'E', 'Ġto', 'ken', 'izer', '.']
[(0, 1), (1, 3), (3, 4), (4, 5), (5, 10), (10, 13), (13, 16), (16, 17), (17, 20), (20, 23), (23, 27), (27, 28)]
 test


In [48]:
# Build a Unigram tokenizer from scratch
unigram_tokenizer = Tokenizer(models.Unigram())

# Add pre-tokenizer
from tokenizers import Regex
unigram_tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFD(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

# Add pre-tokenizer
unigram_tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()
unigram_tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my Unigram tokenizer.")

# Build a trainer
special_tokens = ["<cls>", "<sep>", "<pad>", "<unk>", "<mask>", "<s>", "</s>"]
unigram_trainer = trainers.UnigramTrainer(
    vocab_size=25000, 
    special_tokens=special_tokens,
    unk_token="<unk>",
)
unigram_tokenizer.train_from_iterator(get_training_corpus(), trainer=unigram_trainer)

In [49]:
unigram_encoding = unigram_tokenizer.encode("Let's test my Unigram tokenizer.")
print(unigram_encoding.tokens)

['▁Let', "'", 's', '▁test', '▁my', '▁Un', 'i', 'gram', '▁to', 'ken', 'izer', '.']


In [50]:
cls_token_id = unigram_tokenizer.token_to_id("<cls>")
sep_token_id = unigram_tokenizer.token_to_id("<sep>")
print(cls_token_id, sep_token_id)

0 1


In [51]:
unigram_tokenizer.post_processor = processors.TemplateProcessing(
    single=f"<cls>:0 $A:0 <cls>:2",
    pair=f"$A:0 <sep>:0 $B:1 <cls>:2",
    special_tokens=[
        ("<cls>", cls_token_id),
        ("<sep>", sep_token_id),
    ],
)

In [53]:
unigram_encoding = unigram_tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences!")
print(unigram_encoding.tokens)
print(unigram_encoding.type_ids)

['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.', '.', '.', '<sep>', '▁', 'on', '▁', 'a', '▁pair', '▁of', '▁sentence', 's', '!', '<cls>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]


In [54]:
# Add a MetaSpace decoder
unigram_tokenizer.decoder = decoders.Metaspace()

In [55]:
# Wrap in PreTrainedTokenizerFast
unigram_wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=unigram_tokenizer,
    cls_token="<cls>",
    sep_token="<sep>",
    unk_token="<unk>",
    mask_token="<mask>",
    bos_token="<s>",
    eos_token="</s>",
    padding_side="left",
)